In [84]:
%config Completer.use_jedi = False
import warnings

warnings.filterwarnings(action='ignore')
import tensorflow.compat.v1 as tf

tf.disable_v2_behavior()

학습할 데이터를 만든다.

In [85]:
xData = [i + 1 for i in range(7)]  # 1일 근무시간
yData = [25000, 55000, 75000, 110000, 128000, 155000, 180000]  # 근무시간에 따른 매출금액, 실제값, 레이블, 답, 관측값

과적합을 방지하기 위해서 난수를 발생시켜 가중치(기울기)와 바이어스(y절편)를 정한다.
과적합이란 학습시킨 데이터에서는 높은 정확도를 보이지만 학습시킨 이외의 데이터에서는 낮은 정확도를 보이는 문제점을 말한다.

In [86]:
# tf.random_uniform([난수의 개수])
a = tf.Variable(tf.random_uniform([1], -100, 100))  # 가중치, -100 ~ 100 사이의 난수
b = tf.Variable(tf.random_uniform([1], -100, 100))  # 편향, -100 ~ 100 사이의 난수
print(f'a = {a}\nb = {b}')

a = <tf.Variable 'Variable_21:0' shape=(1,) dtype=float32_ref>
b = <tf.Variable 'Variable_22:0' shape=(1,) dtype=float32_ref>


tensorflow 변수를 초기화하고 난수로 발생시킨 가중치와 바이어스 값을 확인한다.

In [87]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
print(f'a = {sess.run(a)}\nb = {sess.run(b)}')

a = [-92.37118]
b = [76.427155]


근무시간과 매출금액을 기억할 자료형(placeholder)을 선언한다.

In [88]:
x = tf.placeholder(dtype=tf.float32)  # 근무시간을 기억할 placeholder
y = tf.placeholder(dtype=tf.float32)  # 매출금액(실제값)을 기억할 placeholder

1차 방정식 형태의 가설을 세우고 오차(비용, 손실) 함수를 정의한다.

In [89]:
# 1차 방정식 형태의 가설, Y: 예측값
Y = a * x + b
# 오차 함수(평균 제곱법)
# 실제값(yData → y)과 예측값(Y)의 편차에 대한 제곱의 평균을 이용한다.
# square(): 제곱값을 계산한다.
# reduce_mean(): 평균을 계산한다.
cost = tf.reduce_mean(tf.square(Y - y))

경사하강법 함수 오차 함수의 결과를 가장 작게 만드는 방향으로 학습하도록 정의한다.

In [90]:
# 경사하강 알고리즘의 학습율을 설정한다.
learning_rate = tf.Variable(0.01)
# tf.train.GradientDescentOptimizer(): 학습율에 따른 오차 함수를 계산한다.
# minimize(오차함수): 경사하강 알고리즘에 따라 지정된 오차 함수의 최소값을 찾는다.
# optimizer = tf.train.GradientDescentOptimizer(learning_rate)
# train = optimizer.minimize(cost)
train = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

학습시킨다.

In [97]:
# Session을 만들고 변수를 초기화한다.
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for i in range(5001):
    # 오차 함수 결과를 가장 작게 만드는 학습을 할 수 있도록 placeholder에 데이터를 대입하고 학습시킨다.
    # 학습할 데이터는 placeholder에 feed_dict 딕셔너리로 대입한다.
    # feed_dict의 key에는 placeholder 이름을, value에는 placeholder에 저장할 리스트 또는 튜플 타입의 데이터를 지정한다.
    sess.run(train, feed_dict={x: xData, y: yData})

    # 일정한 epoch 단위마다 중간 결과를 출력한다.
    if i % 500 == 0:
        # epoch(i), 오차(cost)
        # print(f'epoch = {i:4d} → loss = {sess.run(cost, feed_dict={x: xData, y: yData}):13.2f}, a = {sess.run(a)[0]:8.2f}, b = {sess.run(b)[0]:7.2f}')
        print('epoch = %4d → 오차 = %13.2f, 가중치 = %8.2f, 바이어스 = %7.2f' % (
        i, sess.run(cost, feed_dict={x: xData, y: yData}), sess.run(a), sess.run(b)))
print(type(sess.run(a)))
print(sess.run(a))

epoch =    0 → 오차 = 4577209856.00, 가중치 = 10409.52, 바이어스 = 2026.53
epoch =  500 → 오차 =    8685604.00, 가중치 = 25538.97, 바이어스 = 1943.00
epoch = 1000 → 오차 =    8633774.00, 가중치 = 25627.72, 바이어스 = 1503.54
epoch = 1500 → 오차 =    8632675.00, 가중치 = 25640.65, 바이어스 = 1439.50
epoch = 2000 → 오차 =    8632653.00, 가중치 = 25642.54, 바이어스 = 1430.16
epoch = 2500 → 오차 =    8632639.00, 가중치 = 25642.81, 바이어스 = 1428.82
epoch = 3000 → 오차 =    8632651.00, 가중치 = 25642.84, 바이어스 = 1428.63
epoch = 3500 → 오차 =    8632656.00, 가중치 = 25642.85, 바이어스 = 1428.61
epoch = 4000 → 오차 =    8632656.00, 가중치 = 25642.85, 바이어스 = 1428.61
epoch = 4500 → 오차 =    8632656.00, 가중치 = 25642.85, 바이어스 = 1428.61
epoch = 5000 → 오차 =    8632656.00, 가중치 = 25642.85, 바이어스 = 1428.61
<class 'numpy.ndarray'>
[25642.848]


8632656.00

In [96]:
for i in range(8, 25):
    print(f'{i:2d}시간 근무를 했을 때 매출 기대금액: {sess.run(Y, feed_dict={x: i})[0]:,.0f}')
sess.close()

 8시간 근무를 했을 때 매출 기대금액: 206,571
 9시간 근무를 했을 때 매출 기대금액: 232,214
10시간 근무를 했을 때 매출 기대금액: 257,857
11시간 근무를 했을 때 매출 기대금액: 283,500
12시간 근무를 했을 때 매출 기대금액: 309,143
13시간 근무를 했을 때 매출 기대금액: 334,786
14시간 근무를 했을 때 매출 기대금액: 360,428
15시간 근무를 했을 때 매출 기대금액: 386,071
16시간 근무를 했을 때 매출 기대금액: 411,714
17시간 근무를 했을 때 매출 기대금액: 437,357
18시간 근무를 했을 때 매출 기대금액: 463,000
19시간 근무를 했을 때 매출 기대금액: 488,643
20시간 근무를 했을 때 매출 기대금액: 514,286
21시간 근무를 했을 때 매출 기대금액: 539,928
22시간 근무를 했을 때 매출 기대금액: 565,571
23시간 근무를 했을 때 매출 기대금액: 591,214
24시간 근무를 했을 때 매출 기대금액: 616,857
